### caption 만들기

1. gemini 사용하여 이미지에 대한 설명 생성
2. clip 임베딩을 위한 형태로 이미지 설명 전처리 

In [7]:
import os
import csv
import re
import google.generativeai as genai
from dotenv import load_dotenv

from PIL import Image
import io

# .env에서 API 키 로드
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Gemini 모델 설정
model = genai.GenerativeModel("gemini-1.5-pro")

# 분석할 이미지들이 들어있는 폴더 경로
# input_folder = rf"C:\Users\mh\Desktop\swbootcamp\project\sandstone_pngs_최종"
input_folder = "./sandstone_최종_진짜"
# 결과 CSV 경로
csv_filename = "output_archive_최종.csv"

# CSV 파일에 헤더 작성 (기존 파일이 없을 때만 작성되도록)
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["input_file", "이미지 설명", "상징적 해석", "우선순위1", "우선순위2", "우선순위3"])


image_files = sorted(
    f for f in os.listdir(input_folder)
    if f.lower().endswith((".png", ".jpg", ".jpeg"))
)

for file_name in image_files[:1]: # 코드 제출용
    image_path = os.path.join(input_folder, file_name)
    print(f"[진행중] {file_name} 분석 중...")

    # 이미지 읽기
    with open(image_path, "rb") as f:
        image_bytes = f.read()

    # 이미지를 PIL로 로드 후 화면에 표시
    # image = Image.open(io.BytesIO(image_bytes))
    # image.show()
    # import hashlib
    # # 이미지 해시 계산 (동일 이미지인지 검증용)
    # image_hash = hashlib.md5(image_bytes).hexdigest()
    # # 처리 시작 로그
    # print(f"[처리시작] 파일명: {file_name} / 이미지 해시: {image_hash}")

    # Gemini API 호출
    response = model.generate_content(
        [
            """
            You are a senior iconographer and semiotician.  
            When you receive an image, first describe every visible geometric element in precise visual terms, then infer the most plausible meanings the pictogram could represent in real-world contexts.  
            — If the image is ambiguous, list multiple ranked interpretations with approximate confidence percentages.  
            — Use clear, concise language; avoid speculation not grounded in visual evidence.  
            — Your final answer must include:  
            1. Pixel-level description of shapes, lines, colors, relative positions, and negative space.  
            2. Mapping of those visual primitives to common symbolic conventions (e.g., “circle with wedge = power button”).  
            3. Up to three possible semantics ranked by likelihood, each with a one-sentence rationale.  
            4. If confidence < 40 %, ask follow-up questions that could disambiguate.  
            All measurements can be approximate.


            모든 응답은 아래 형식으로 반드시 출력해줘.
        
            이미지 설명:
            [여기에 이미지 설명]

            상징적 해석:
            [여기에 상징적 해석]

            가능한 의미:
            1. [우선순위1]
            2. [우선순위2]
            3. [우선순위3]

            """,
            {
                "mime_type": "image/png",
                "data": image_bytes,
            },
            "이 이미지를 분석해줘. 답변은 한글로 해줘. 이미지는 다양한 기능과 상태를 나타내는 심볼 중 하나로, 주로 디지털 기기나 소프트웨어에서 사용돼."
        ]
    )

    # Gemini 응답에서 텍스트
    response_text = response.text
    print(response_text)

    desc_match = re.search(r"이미지 설명:\s*(.+?)(?=\n\s*상징적 해석:|\Z)", response_text, re.S)
    image_desc = desc_match.group(1).strip() if desc_match else ""

    # 상징적 해석 파싱
    symbolic_match = re.search(r"상징적 해석:\s*(.+?)(?=\n\s*가능한 의미:|\Z)", response_text, re.S)
    symbolic_interp = symbolic_match.group(1).strip() if symbolic_match else ""

    # 가능한 의미 파싱 (1~3)
    priority_matches = re.findall(r"\d+\.\s*(.+?)(?=\n\d+\.|\Z)", response_text, re.S)

    priority1, priority2, priority3 = "", "", ""
    if len(priority_matches) >= 1:
        priority1 = priority_matches[0].strip()
    if len(priority_matches) >= 2:
        priority2 = priority_matches[1].strip()
    if len(priority_matches) >= 3:
        priority3 = priority_matches[2].strip()
        
    # CSV에 결과 누적 저장
    with open(csv_filename, mode="a", newline="", encoding="utf-8-sig") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([file_name, image_desc, symbolic_interp, priority1, priority2, priority3])

    print(f"[완료] {file_name} 결과 저장 완료!")

print(f"\n[전체 완료] 결과는 {csv_filename} 파일에 누적 저장되었습니다.")


[진행중] accessibility.png 분석 중...
이미지 설명:
이 아이콘은 검은색 외곽선으로 그려진 단순화된 사람 형태를 보여줍니다. 머리는 원으로, 몸통과 팔은 수평선과 연결된 두 개의 곡선으로, 다리는 아래쪽으로 벌어진 두 개의 곡선으로 표현되어 있습니다. 모든 선은 일정한 두께를 가지며, 내부는 흰색으로 비어 있습니다. 사람 형태는 대칭적이며 중앙에 위치해 있습니다. 배경은 흰색입니다.

상징적 해석:
원은 머리, 수평선과 연결된 곡선은 팔을 벌린 몸통, 아래쪽으로 벌어진 곡선은 다리를 나타냅니다. 팔을 벌린 자세는 개방성, 환영, 또는 활동적인 움직임을 암시할 수 있습니다. 단순화된 형태는 보편성과 접근성을 나타냅니다.

가능한 의미:
1. 사람(90%): 단순화된 인간 형태는 가장 직관적으로 사람, 사용자 또는 개인을 나타냅니다.
2. 활동 또는 움직임(60%): 팔과 다리가 벌어진 자세는 걷기, 달리기, 춤추기, 또는 기타 신체 활동을 나타낼 수 있습니다.
3. 접근성 또는 개방성(50%): 팔을 벌린 자세는 환영, 포용, 또는 접근성을 상징할 수도 있습니다. 특히 접근성 설정과 관련된 기능을 나타낼 가능성이 있습니다.



[완료] accessibility.png 결과 저장 완료!

[전체 완료] 결과는 output_archive_최종.csv 파일에 누적 저장되었습니다.


In [ ]:
#전처리
#output_archive.csv

import pandas as pd

df = pd.read_csv("output_archive_최종.csv")

df.head()

,input_file,이미지 설명,상징적 해석,우선순위1,우선순위2,우선순위3
0,accessibility.png,이 아이콘은 검은색 외곽선으로 그려진 단순화된 사람 형상을 묘사합니다. 원형 머리...,"원은 머리를 나타내는 일반적인 상징입니다. 뻗은 팔과 다리는 활동, 개방성, 또는 ...",사람 또는 사용자의 활동 또는 움직임을 나타내는 아이콘 (70%): 팔과 다리가 뻗...,접근성 또는 개방성을 나타내는 아이콘 (60%): 팔을 벌린 자세는 환영하거나 포용...,사람의 위치 또는 현재 상태를 나타내는 아이콘 (50%): 지도 또는 위치 기반 서...
1,ai.png,이 이미지는 두 개의 동일한 루프가 중앙에서 교차하는 형태를 하고 있습니다. 각 루...,"이 이미지는 겹쳐진 두 개의 루프로 인해 무한대 기호(∞)를 연상시키지만, 형태가 ...",**연결 또는 상호작용(70%):** 두 개의 루프가 서로 엮여 있는 모습은 두 개...,"**무한 또는 지속성(60%):** 루프 형태는 순환이나 반복을 연상시키며, 이는 ...",**균형 또는 조화(50%):** 중앙의 마름모 형태와 전체적인 대칭성은 균형이나 ...
2,alert01.png,이 아이콘은 검은색 윤곽선으로 그려진 종의 단순화된 형태를 보여줍니다. 종의 윗부분...,"종 모양은 일반적으로 알림, 경고, 신호 등을 상징합니다. 원형 고리는 종을 울리...","알림 (95%): 종은 전통적으로 알림이나 신호의 도구로 사용되었으며, 디지털 환경...",경고 (80%): 상황에 따라 종소리는 위험이나 주의를 요하는 상황을 알리는 경고의...,알람 (75%): 종소리는 특정 시간을 알리거나 기상을 돕는 알람 기능으로도 활용될...
3,alert02.png,"검은색 실루엣으로 표현된 종 모양. 상단은 둥글게, 하단은 좁아지는 형태이며, 아래...","종은 소리를 내는 도구로 알림, 경고, 시작, 종료 등을 상징한다. 단순화된 형태와...","알림 (95%): 디지털 환경에서 새로운 메시지, 업데이트, 이벤트 등의 알림을 ...",경고 (4%): 긴급 상황이나 주의가 필요한 사항을 알리는 경고의 의미를 가질 수...,소리/음량 (1%): 소리 또는 음량 조절과 관련된 기능을 나타낼 가능성도 있지만...
4,appscontents.png,이 아이콘은 검은색 윤곽선으로 그려진 세 개의 사각형으로 구성되어 있다. 가장 위쪽...,- 위쪽 사각형과 삼각형은 비디오 또는 오디오 콘텐츠의 재생을 나타내는 표준적인 기...,"(90% 확률) 비디오 플레이어 인터페이스의 일부로, 현재 재생 중인 비디오와 메뉴...",(70% 확률) 멀티미디어 프레젠테이션 도구의 컨트롤 패널을 나타낸다. 재생 버튼은...,(40% 확률) 온라인 교육 플랫폼의 네비게이션 요소. 재생 버튼은 강의 비디오 ...


In [ ]:
#df의 우선순위1 컬럼에서 90% 처럼 확률 표현 전부 삭제

import re

# (숫자% 확률) 형태와 (숫자%) 형태 모두 삭제
if '우선순위1' in df.columns:
    df['우선순위1'] = df['우선순위1'].astype(str).apply(lambda x: re.sub(r'\(\d+%[^)]*\)', '', x))  # (90% 확률) 등
    df['우선순위1'] = df['우선순위1'].astype(str).apply(lambda x: re.sub(r'\d+%', '', x))  # 90% 등
    df['우선순위1'] = df['우선순위1'].str.replace(r'^\s+|\s+$', '', regex=True)  # 앞뒤 공백 제거
    df['우선순위1'] = df['우선순위1'].str.replace(r':\s*$', '', regex=True)  # 끝에 남은 콜론 제거
    # ** 제거
    df['우선순위1'] = df['우선순위1'].str.replace(r'\*\*', '', regex=True)
    # " 제거
    df['우선순위1'] = df['우선순위1'].str.replace(r'"', '', regex=True)
    # 77자 이내로 제거
    df['우선순위1'] = df['우선순위1'].str.slice(0, 30)
    print('우선순위1 컬럼에서 확률 표현 삭제 완료!')
    display(df[['input_file', '우선순위1']].head())
else:
    print('우선순위1 컬럼이 존재하지 않습니다.')

우선순위1 컬럼에서 확률 표현 삭제 완료!


,input_file,우선순위1
0,accessibility.png,사람 또는 사용자의 활동 또는 움직임을 나타내는 아이콘
1,ai.png,연결 또는 상호작용: 두 개의 루프가 서로 엮여 있는
2,alert01.png,알림 : 종은 전통적으로 알림이나 신호의 도구로 사용되
3,alert02.png,"알림 : 디지털 환경에서 새로운 메시지, 업데이트,"
4,appscontents.png,"비디오 플레이어 인터페이스의 일부로, 현재 재생 중인"


In [ ]:

df.head()
df.to_csv("output_archive_processed_최종.csv", index=False, encoding="utf-8-sig")

In [ ]:
df.우선순위1.values[:10]

array(['사람 또는 사용자의 활동 또는 움직임을 나타내는 아이콘', '연결 또는 상호작용: 두 개의 루프가 서로 엮여 있는 ',
       '알림 : 종은 전통적으로 알림이나 신호의 도구로 사용되', '알림 :  디지털 환경에서 새로운 메시지, 업데이트, ',
       '비디오 플레이어 인터페이스의 일부로, 현재 재생 중인 ', '다음 단계 또는 다음 페이지로 이동 : 오른쪽 방향 화',
       '되돌리기/이전 작업 실행 취소 : 왼쪽 화살표와 반원의', '공유하기 :  곡선 형태의 화살표는 콘텐츠를 다른 곳으',
       '아래쪽 화살표 기능 : 디지털 인터페이스에서 아래쪽 스', '사용자 인터페이스에서 이전 버튼이나 뒤로 기능을 나타냅'],
      dtype=object)

### (추가) 이미지 성능 이슈 체크

In [ ]:
from PIL import Image, ImageChops
import os

def compare_images(img_path1, img_path2):
    # 파일 크기 비교
    size1 = os.path.getsize(img_path1)
    size2 = os.path.getsize(img_path2)
    print(f"파일 크기: {img_path1} = {size1} bytes")
    print(f"파일 크기: {img_path2} = {size2} bytes\n")

    # 이미지 열기
    img1 = Image.open(img_path1)
    img2 = Image.open(img_path2)

    # 해상도 및 모드 비교
    print(f"{img_path1} 해상도: {img1.size}, 모드: {img1.mode}")
    print(f"{img_path2} 해상도: {img2.size}, 모드: {img2.mode}\n")

    # 이미지 크기 다르면 경고
    if img1.size != img2.size:
        print("경고: 두 이미지 해상도가 다릅니다. 픽셀 차이 계산 불가능할 수 있음.\n")

    # 동일 크기여야 픽셀 차이 계산 가능
    if img1.size == img2.size:
        # 두 이미지 차이 계산 (절대값)
        diff = ImageChops.difference(img1, img2)
        diff = diff.convert("L")  # 흑백 변환

        # 차이 픽셀 수
        diff_pixels = sum(diff.histogram()[1:])
        total_pixels = img1.size[0] * img1.size[1]

        diff_ratio = diff_pixels / total_pixels * 100
        print(f"픽셀 차이: {diff_pixels} / {total_pixels} = {diff_ratio:.2f}%")

        # 차이 이미지 저장 (선택)
        diff.save("diff_image.png")
        print("차이 이미지를 diff_image.png로 저장했습니다.")

        img1_rgb = img1.convert("RGB")
        img2_rgb = img2.convert("RGB")
        diff = ImageChops.difference(img1_rgb, img2_rgb).convert("L")
        diff.save("diff_rgb.png")

    else:
        print("이미지 크기가 다르므로 픽셀 단위 차이 계산을 생략합니다.")

# 사용 예
compare_images(rf"C:\Users\mh\Desktop\swbootcamp\Archive\accessibility.png", 
rf"C:\Users\mh\Desktop\swbootcamp\project\sandstone_pngs_최종\accessibility.png")


파일 크기: C:\Users\mh\Desktop\swbootcamp\Archive\accessibility.png = 60004 bytes
파일 크기: C:\Users\mh\Desktop\swbootcamp\project\sandstone_pngs_최종\accessibility.png = 23100 bytes

C:\Users\mh\Desktop\swbootcamp\Archive\accessibility.png 해상도: (1194, 1075), 모드: RGBA
C:\Users\mh\Desktop\swbootcamp\project\sandstone_pngs_최종\accessibility.png 해상도: (1194, 1075), 모드: RGBA

픽셀 차이: 1090924 / 1283550 = 84.99%
차이 이미지를 diff_image.png로 저장했습니다.


### (추가) 파일명 전처리

In [ ]:
import os
from natsort import natsorted
# 폴더 경로 설정 (절대경로 또는 상대경로 사용 가능)
folder_a = 'sandstone_최종'  # 이름을 바꿀 파일들이 있는 폴더
folder_b = './project/sandstone_pngs_최종'  # 참조할 이름이 있는 폴더

# 자연 정렬 사용 (Windows 탐색기와 동일한 순서)
files_a = natsorted(os.listdir(folder_a))
files_b = natsorted(os.listdir(folder_b))

# 파일 수 확인
if len(files_a) != len(files_b):
    raise ValueError("두 폴더의 파일 개수가 다릅니다.")

# 파일 이름 변경 실행
for file_a, file_b in zip(files_a, files_b)[:10]:
    a_path = os.path.join(folder_a, file_a)

    # 확장자는 폴더 A의 파일 확장자 유지
    name_b, _ = os.path.splitext(file_b)
    _, ext_a = os.path.splitext(file_a)

    new_name = name_b + ext_a
    new_path = os.path.join(folder_a, new_name)

    os.rename(a_path, new_path)
    print(f"✔ {file_a} → {new_name}")

# # 🔍 미리 보기 목록 생성
# rename_plan = []
# print("🔍 미리보기: 다음과 같이 이름이 변경됩니다\n")

# for file_a, file_b in zip(files_a, files_b):
#     name_b, _ = os.path.splitext(file_b)
#     _, ext_a = os.path.splitext(file_a)
#     new_name = name_b + ext_a
#     rename_plan.append((file_a, new_name))
#     print(f"  {file_a}  →  {new_name}")

# # ✅ 사용자 확인
# confirm = input("\n✅ 위대로 이름을 변경하시겠습니까? (y/n): ").strip().lower()
# if confirm == 'y':
#     for file_a, new_name in rename_plan:
#         old_path = os.path.join(folder_a, file_a)
#         new_path = os.path.join(folder_a, new_name)
#         os.rename(old_path, new_path)
#     print("\n✅ 이름 변경이 완료되었습니다.")
# else:
#     print("\n🚫 이름 변경이 취소되었습니다.")    
